In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import to_categorical, plot_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
train_path='/content/drive/My Drive/Colab Notebooks/ANN_CNN_chest_Xray_Pneumonia_detection/chest_xray/train'
test_path='/content/drive/My Drive/Colab Notebooks/ANN_CNN_chest_Xray_Pneumonia_detection/chest_xray/test'

In [5]:
pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# import required module
from PIL import Image
  
# get image
filepath = "/content/drive/My Drive/Colab Notebooks/ANN_CNN_chest_Xray_Pneumonia_detection/chest_xray/test/NORMAL/IM-0001-0001.jpeg"
img = Image.open(filepath)

In [7]:
# get width and height
width = img.width
height = img.height

In [8]:
# display width and height
print("The height of the image is: ", height)
print("The width of the image is: ", width)

The height of the image is:  1317
The width of the image is:  1857


In [9]:
import os

In [10]:
print(os.listdir(train_path))

['PNEUMONIA', 'NORMAL']


In [11]:
print(os.listdir(test_path))

['NORMAL', 'PNEUMONIA']


In [48]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.25)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [52]:
# Preprocess all train images
train_generator = train_data_gen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/ANN_CNN_chest_Xray_Pneumonia_detection/chest_xray/train',
        target_size=(48, 48),
        batch_size=32,
        color_mode="grayscale",
        class_mode='categorical',
        subset='training')

Found 3913 images belonging to 2 classes.


In [53]:
# Preprocess all test images
test_generator = train_data_gen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/ANN_CNN_chest_Xray_Pneumonia_detection/chest_xray/train',
        target_size=(48, 48),
        batch_size=32,
        color_mode="grayscale",
        class_mode='categorical',
        subset='validation')

Found 1303 images belonging to 2 classes.


In [54]:
# create model structure
from keras.models import Sequential
pheumonia_detection_model = Sequential()

In [55]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
pheumonia_detection_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
pheumonia_detection_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
pheumonia_detection_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
pheumonia_detection_model.add(MaxPooling2D(pool_size=(3, 3)))
pheumonia_detection_model.add(Dropout(0.35))

In [56]:
pheumonia_detection_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
pheumonia_detection_model.add(MaxPooling2D(pool_size=(3, 3)))
pheumonia_detection_model.add(Dropout(0.35))
pheumonia_detection_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
pheumonia_detection_model.add(MaxPooling2D(pool_size=(2, 2)))
pheumonia_detection_model.add(Dropout(0.35))

In [57]:
pheumonia_detection_model.add(Flatten())
pheumonia_detection_model.add(Dense(1024, activation='relu'))
pheumonia_detection_model.add(Dropout(0.35))
pheumonia_detection_model.add(Dense(2, activation='softmax'))

In [58]:
pheumonia_detection_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 conv2d_18 (Conv2D)          (None, 44, 44, 64)        18496     
                                                                 
 conv2d_19 (Conv2D)          (None, 42, 42, 128)       73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 14, 14, 128)       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 12, 12, 256)       295168    
                                                      

In [59]:
import cv2
cv2.ocl.setUseOpenCL(False)

In [60]:
from tensorflow.keras.optimizers import Adam

In [61]:
pheumonia_detection_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [62]:
# Train the neural network/model
pheumonia_model_info = pheumonia_detection_model.fit(
        train_generator,
        steps_per_epoch=5216 // 64,
        epochs=20,
        validation_data=test_generator,
        validation_steps=624 // 64)

Epoch 1/20
81/81 [==============================] - 81s 991ms/step - loss: 0.5773 - accuracy: 0.7408 - val_loss: 0.5516 - val_accuracy: 0.7222
Epoch 2/20
81/81 [==============================] - 78s 958ms/step - loss: 0.3936 - accuracy: 0.8187 - val_loss: 0.3416 - val_accuracy: 0.8368
Epoch 3/20
81/81 [==============================] - 75s 927ms/step - loss: 0.2550 - accuracy: 0.8949 - val_loss: 0.3140 - val_accuracy: 0.8681
Epoch 4/20
81/81 [==============================] - 77s 949ms/step - loss: 0.1759 - accuracy: 0.9309 - val_loss: 0.1978 - val_accuracy: 0.9271
Epoch 5/20
81/81 [==============================] - 75s 925ms/step - loss: 0.1359 - accuracy: 0.9475 - val_loss: 0.3297 - val_accuracy: 0.8576
Epoch 6/20
81/81 [==============================] - 78s 959ms/step - loss: 0.1215 - accuracy: 0.9514 - val_loss: 0.1566 - val_accuracy: 0.9410
Epoch 7/20
81/81 [==============================] - 77s 952ms/step - loss: 0.0995 - accuracy: 0.9599 - val_loss: 0.1040 - val_accuracy: 0.9688